In [ ]:
# Homework Reflection 9 Question 1
# Write some code that will use a simulation to estimate the standard deviation of the coefficient when there is heteroskedasticity.  
# Compare these standard errors to those found via statsmodels OLS or a similar linear regression model.

In [4]:
import numpy as np
import statsmodels.api as sm


np.random.seed(0)
n = 1000        
reps = 500      
DO_COVERAGE = True  


betas = []
se_ols = []
se_hc1 = []
se_hc3 = []


for _ in range(reps):
   
    X = np.random.normal(0, 1, n)
    Z = np.random.normal(0, 1, n)  

    
    eps = np.random.normal(0, 1 + np.abs(X), n)

   
    Y = 2*X + 1*Z + eps

    
    Xmat = sm.add_constant(np.column_stack([X, Z]))
    m = sm.OLS(Y, Xmat).fit()

   
    betas.append(m.params[1])                     
    se_ols.append(m.bse[1])                       
    se_hc1.append(m.get_robustcov_results(cov_type='HC1').bse[1])  
    se_hc3.append(m.get_robustcov_results(cov_type='HC3').bse[1])  


betas = np.array(betas)
se_ols = np.array(se_ols)
se_hc1 = np.array(se_hc1)
se_hc3 = np.array(se_hc3)


emp_sd = betas.std(ddof=1)
avg_ols = se_ols.mean()
avg_hc1 = se_hc1.mean()
avg_hc3 = se_hc3.mean()

ratio_ols   = avg_ols / emp_sd
ratio_hc1   = avg_hc1 / emp_sd
ratio_hc3   = avg_hc3 / emp_sd
under_ols   = 100 * (1 - ratio_ols)
under_hc1   = 100 * (1 - ratio_hc1)
under_hc3   = 100 * (1 - ratio_hc3)


print(f"Empirical SD of beta_X      : {emp_sd:.4f}")
print(f"Avg OLS SE (assumes homosk.) : {avg_ols:.4f}  "
      f"(ratio={ratio_ols:.3f}, under by {under_ols:.1f}%)")
print(f"Avg Robust SE (HC1)         : {avg_hc1:.4f}  "
      f"(ratio={ratio_hc1:.3f}, under by {under_hc1:.1f}%)")
print(f"Avg Robust SE (HC3)         : {avg_hc3:.4f}  "
      f"(ratio={ratio_hc3:.3f}, under by {under_hc3:.1f}%)")

Empirical SD of beta_X      : 0.0877
Avg OLS SE (assumes homosk.) : 0.0600  (ratio=0.684, under by 31.6%)
Avg Robust SE (HC1)         : 0.0843  (ratio=0.961, under by 3.9%)
Avg Robust SE (HC3)         : 0.0847  (ratio=0.966, under by 3.4%)


In [ ]:
# Lets just say we are trying to guess how heavy a bag is and we it 500 times, the guesses jump around 
# Emperical SD means how much the estimate varies when we repeat the measurement every time 
# Emperical SD - the average amoount of the variation 
# OLS SE = 0.06 it is off by 32% - being overconfindent - making us think that we are more precise to the actual weight than in reality what it is 
# Robust SE - almost matches the prediction it is just off by 3 - 4%
# Robust SE - gives a more honest answer when the data is uneven 

In [ ]:
# Homework Reflection 9 Question 2
# Write some code that will use a simulation to estimate the standard deviation of the coefficient when errors are highly correlated / non-independent.
# Compare these standard errors to those found via statsmodels OlS or a similar linear regression model.


In [5]:
import numpy as np
import statsmodels.api as sm

np.random.seed(0)
n, reps, rho = 1000, 500, 0.9

betas, se_ols, se_hac = [], [], []

for _ in range(reps):
    X = np.random.normal(0, 1, n)

    
    e = np.zeros(n)
    for t in range(1, n):
        e[t] = rho * e[t-1] + np.random.normal()

    Y = 2*X + e

    m = sm.OLS(Y, sm.add_constant(X)).fit()
    betas.append(m.params[1])
    se_ols.append(m.bse[1])
    se_hac.append(m.get_robustcov_results(cov_type='HAC', maxlags=1).bse[1])

betas, se_ols, se_hac = map(np.array, (betas, se_ols, se_hac))

print("Empirical SD :", betas.std(ddof=1))
print("Avg OLS SE   :", se_ols.mean())
print("Avg HAC SE   :", se_hac.mean())


Empirical SD : 0.07012066915541845
Avg OLS SE   : 0.07156014914836889
Avg HAC SE   : 0.07122063189469006


In [ ]:
# When the experiment is repeated many times the slop changes - Emperical SD 
# OLS SE – the regular method – 0.0716
# HAC SE – the autocorrelation-robust method – 0.0712
# In this case the normal method and the robust method both did a good job in making the prediction



In [ ]:
# Show that if the correlation between coefficients is high enough, then the estimated standard deviation of the coefficient, using bootstrap errors, 
# might not match that found by a full simulation of the Data Generating Process.  (This can be fixed if you have a huge amount of data for the bootstrap simulation.)

In [7]:
import numpy as np
import statsmodels.api as sm

np.random.seed(42)


n = 500            
reps = 500         
boot_reps = 500    
rho = 0.99         
beta1, beta2 = 2.0, 3.0
sigma = 1.0

def gen_X(n, rho):
    x1 = np.random.normal(0, 1, n)
    u  = np.random.normal(0, 1, n)
    x2 = rho * x1 + np.sqrt(max(1 - rho**2, 1e-12)) * u
    return x1, x2


beta1_full = []
for _ in range(reps):
    x1, x2 = gen_X(n, rho)
    y = beta1 * x1 + beta2 * x2 + np.random.normal(0, sigma, n)
    X = sm.add_constant(np.column_stack([x1, x2]))
    beta1_full.append(sm.OLS(y, X).fit().params[1])
emp_sd = np.std(beta1_full, ddof=1)


x1_fix, x2_fix = gen_X(n, rho)
y_fix = beta1 * x1_fix + beta2 * x2_fix + np.random.normal(0, sigma, n)
X_fix = sm.add_constant(np.column_stack([x1_fix, x2_fix]))
m_fix = sm.OLS(y_fix, X_fix).fit()
resid = m_fix.resid
yhat  = m_fix.fittedvalues

beta1_boot = []
for _ in range(boot_reps):
    y_boot = yhat + np.random.choice(resid, size=n, replace=True)
    beta1_boot.append(sm.OLS(y_boot, X_fix).fit().params[1])
boot_sd = np.std(beta1_boot, ddof=1)


print(f"Full DGP SD of beta1     : {emp_sd:.4f}   (ground truth)")
print(f"Bootstrap SD of beta1    : {boot_sd:.4f}   (resample residuals, X fixed)")
print(f"Bootstrap / Full ratio   : {boot_sd/emp_sd:.3f}  (want ≈ 1.00)")

Full DGP SD of beta1     : 0.3087   (ground truth)
Bootstrap SD of beta1    : 0.2903   (resample residuals, X fixed)
Bootstrap / Full ratio   : 0.941  (want ≈ 1.00)


In [ ]:
# The true amount the number moves around is 0.3087
# The bootstrap method says it moves 0.2903
# Bootstrap is a little too low because it didn’t let X change
# With a lot more data, they would match

In [ ]:
# Week 11 Reflection Question 1 
# Construct a dataset for an event study where the value, derivative, and second derivative of a trend all change discontinuously (suddenly) after an event.
# Build a model that tries to decide whether the event is real (has a nonzero effect) using:
# (a) only the value,
# (b) the value, derivative, and second derivative.
# Which of these models is better at detecting and/or quantifying the impact of the event?  (What might "better" mean here?)

In [ ]:
import numpy as np
import statsmodels.api as sm

np.random.seed(0)


T, t0 = 200, 100
t = np.arange(T).astype(float)
D = (t >= t0).astype(float)          
tau = t - t0                        


L, S, C = 1.0, 0.06, 0.001           
y = 0.0 + 0.02*t - 0.0001*t**2 \
    + L*D + S*tau*D + C*(tau**2)*D \
    + np.random.normal(0, 0.3, T)


m_a = sm.OLS(y, sm.add_constant(D)).fit()


Xb = np.column_stack([
    np.ones(T), t, t**2, D, D*tau, D*(tau**2)
])
m_b = sm.OLS(y, Xb).fit()

print("One dataset:")
print(f"(a) value-only, p(D) = {m_a.pvalues[1]:.4g}")


R = np.zeros((3, Xb.shape[1])); R[0,3]=R[1,4]=R[2,5]=1
p_joint = float(m_b.f_test((R, np.zeros(3))).pvalue)
print(f"(b) value+slope+curvature, joint p = {p_joint:.4g}")


reps = 200
hits_a = hits_b = 0
for _ in range(reps):
    eps = np.random.normal(0, 0.3, T)
    y_sim = 0.0 + 0.02*t - 0.0001*t**2 + L*D + S*tau*D + C*(tau**2)*D + eps

    pa = sm.OLS(y_sim, sm.add_constant(D)).fit().pvalues[1]
    hits_a += (pa < 0.05)

    mb = sm.OLS(y_sim, Xb).fit()
    p_joint = float(mb.f_test((R, np.zeros(3))).pvalue)
    hits_b += (p_joint < 0.05)

print("\nDetection rate (power, α=0.05) over", reps, "runs:")
print(f"(a) value-only               : {hits_a/reps:.2f}")
print(f"(b) value+slope+curvature    : {hits_b/reps:.2f}")

One dataset:
(a) value-only, p(D) = 1.491e-39
(b) value+slope+curvature, joint p = 1.953e-42

Detection rate (power, α=0.05) over 200 runs:
(a) value-only               : 1.00
(b) value+slope+curvature    : 1.00


In [ ]:
# In all 200 tests, both models spotted the event every time 
# Model a - measures sudden jump 
# model b - measure the jump, change in speed, and the change in curvature 
# Model b - gives a more accurate and complete picture 
# Model b is better because it account for different factors 

In [ ]:
# Week 11 Reflection Question 2 
# Construct a dataset in which there are three groups whose values each increase discontinuously (suddenly) by the same amount at a shared event; they change in parallel
# over time, but they have different starting values.  Create a model that combines group fixed effects with an event study, as suggested in the online reading.
# Explain what you did, how the model works, and how it accounts for both baseline differences and the common event effect.

In [ ]:
import numpy as np
import pandas as pd
import statsmodels.formula.api as smf

np.random.seed(0)


G = ['A','B','C']
T = 40
t0 = 20                     
jump = 1.5                  
slope = 0.08                
baseline = {'A':0.0, 'B':2.0, 'C':-1.0}

rows = []
for g in G:
    for t in range(T):
        post = 1 if t >= t0 else 0
        y = baseline[g] + slope*t + jump*post + np.random.normal(0, 0.3)
        rows.append((g, t, post, y))

df = pd.DataFrame(rows, columns=['group','t','post','y'])
df['group'] = df['group'].astype('category')  


model = smf.ols('y ~ group + t + post', data=df).fit()
print(model.summary().tables[1])


print("\nMean change (post - pre) by group:")
print(df.groupby('group').apply(lambda d: d.loc[d.post==1,'y'].mean() - d.loc[d.post==0,'y'].mean()))

                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0745      0.071      1.055      0.294      -0.065       0.214
group[T.B]     1.7941      0.067     26.861      0.000       1.662       1.926
group[T.C]    -0.9561      0.067    -14.314      0.000      -1.088      -0.824
t              0.0841      0.005     17.786      0.000       0.075       0.093
post           1.3783      0.109     12.625      0.000       1.162       1.595

Mean change (post - pre) by group:
group
A    2.945925
B    3.112874
C    3.122626
dtype: float64


/tmp/ipykernel_9417/3065229535.py:32: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  print(df.groupby('group').apply(lambda d: d.loc[d.post==1,'y'].mean() - d.loc[d.post==0,'y'].mean()))
/tmp/ipykernel_9417/3065229535.py:32: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  print(df.groupby('group').apply(lambda d: d.loc[d.post==1,'y'].mean() - d.loc[d.post==0,'y'].mean()))


In [ ]:
# I made three groups: A, B, C 
# Each group starts at a different base line level (height)
# In addition, random noise was added 
# Formula y ~ group + t + post
# Group - each group, starts at its own baseline level 
# t = time - to track the sready rise over time 
# post = measure the jump after time 
# group part removes the differences at the starting point 
# event part find the average size of the shared jump 

In [ ]:
# Week 12 Reflection 
# Construct a dataset in which prior trends do not hold, and in which this makes the differences-in-differences come out wrong.  Explain why the
# differences-in-differences estimate of the effect comes out higher or lower than the actual effect.

In [ ]:
import numpy as np
import pandas as pd

np.random.seed(0)


pre_trend_treated = 1.0   
pre_trend_control = 0.0   
true_effect = 2.0         


mean_ctl_pre = 5
mean_ctl_post = mean_ctl_pre + pre_trend_control  # flat
mean_trt_pre = 6
mean_trt_post = mean_trt_pre + pre_trend_treated + true_effect


df = pd.DataFrame({
    "group": ["control","control","treated","treated"],
    "period": ["pre","post","pre","post"],
    "mean_y": [mean_ctl_pre, mean_ctl_post, mean_trt_pre, mean_trt_post]
})


change_ctl = mean_ctl_post - mean_ctl_pre
change_trt = mean_trt_post - mean_trt_pre
did = change_trt - change_ctl

print(df)
print("\nControl change:", change_ctl)
print("Treated change:", change_trt)
print("DiD estimate  :", did)
print("True effect   :", true_effect)

     group period  mean_y
0  control    pre     5.0
1  control   post     5.0
2  treated    pre     6.0
3  treated   post     9.0

Control change: 0.0
Treated change: 3.0
DiD estimate  : 3.0
True effect   : 2.0


In [ ]:
# True Effect = 2 - the actual event made the treatment group go up by 2 points 
# DiD estimate is 3 
# Before the event the treated group was going up but control group did not experience a change 
# DiD thought they would both move even without an event that is why it is estimated 3 but in reality that was not the case - DiD overestimated 